In [ ]:
/***************************
编写自定义函数：
    标量值函数（返回但当个值）
    表值函数（返回一个查询结果）
****************************/

--select Sum(CardMoney) form BankCard
--编写一个函数，求银行金额的总和
create function GetSumMoney()/*传入参数*/ returns money --返回值 注意是returns
as
begin 
    declare @sum money
    select @sum = Sum(CardMoney) from BankCard
    return @sum 
end 
--函数的调用
select dbo.GetSumMoney() --用select展示出来

--传入账户的编号，返回用户名
create function GetRealNameById(@accid int) returns varchar(30)
as
begin
    declare @name varchar(30)
    select RealName from AccountInifo where AccountId = @accid
    return @name
end

select dbo.GetRealNameById(2)

--传递开始和结束时间，返回交易记录（存钱和取钱）返回表
--方案一
create function GetRecordByTime(@start varchar(30),@end varchar(30))
returns @result table --return 是这个查询结果的名字，建查询结果给这个result
(
    /*这里是建立表的结构，就像建立一个新表一样*/
    RealName varchar(20),
    CardNo varchar(30),
    MoneyInBank money,
    MoneyOutBank money,
    ExchangeTime smalldatetime 
)
as 
begin 
    insert into @result --像我们上面结构的表添加数据，数据为下面查询的数据
    select RealName 姓名, CardExchange.CardNo 卡号,MoneyInBank 存钱金额,MoneyOutBank 取钱金额
    ExchangeTime 交易时间  from CardExchange 
    inner join BankCard on CardExchange.cardNo = BankCard.cardNo
    inner join AccountInifo on BankCard.AccountId = AccountInifo.AccountID
    where ExchangeTime between @start+'00:00:00' and @end+'23:59:59' 
    return  --作为一个返回值返回
end

--调用函数，这里要用select *，因为函数返回结果是一个表
select * from GetRecordByTime('2022-1-1','2022-5-6')

--方案二
/*不规定返回的表结构直接将查询结果返回*/
create function GetRecordByTime(@start varchar(30),@end varchar(30))
return table
as  
    return 
    select RealName 姓名, CardExchange.CardNo 卡号,MoneyInBank 存钱金额,MoneyOutBank 取钱金额
    ExchangeTime 交易时间  from CardExchange 
    inner join BankCard on CardExchange.cardNo = BankCard.cardNo
    inner join AccountInifo on BankCard.AccountId = AccountInifo.AccountID
    where ExchangeTime between @start+'00:00:00' and @end+'23:59:59' 
go

/*方案二使用局限性：函数体内只能有return+sql查询结果（里面不能创建变量），不能处理复杂逻辑*/
